# Projeto 1 - Ciência dos Dados

Nome: Gustavo Marques Borges 

Nome: Luiz Eduardo Correa Santoro

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk
from nltk.tokenize import TweetTokenizer
import re 
# nltk.download('stopwords') dowlod stopword Corpus

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'Magazine_Luiza_Final.xlsx'
if filename in os.listdir():
   print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
   print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Magazine_Luiza_Final.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename, coverters={'Tweets': str})
train['Classificação'] = train['Classificação'].astype('category')
train.Classificação.cat.categories = ['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante']
train.Classificação.cat = pd.Categorical(train.Classificação, categories=['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante'], ordered=True)
train.head(5)

,Tweets,Classificação
0,@tavaresqq a sua troca deu certo? precisando é...,Muito Irrelevante
1,o pessoal da #magazineluiza ja me conhece de ...,Relevante
2,"@magazineluiza bom dia, cadê o meu retorno da ...",Muito Relevante
3,oi @magazineluiza o @luansantana tá há dois m...,Muito Irrelevante
4,@grondinellys @magazineluiza você só precisa d...,Irrelevante


In [4]:
filtra_Train_MI = train['Classificação'] == 'Muito Irrelevante'
Train_MI = train.loc[filtra_Train_MI,:] 

filtra_Train_I = train['Classificação'] == 'Irrelevante'
Train_I = train.loc[filtra_Train_I,:]

filtra_Train_N = train['Classificação'] == 'Nulo'
Train_N = train.loc[filtra_Train_N,:]

filtra_Train_R = train['Classificação'] == 'Relevante'
Train_R = train.loc[filtra_Train_R,:]

filtra_Train_MR = train['Classificação'] == 'Muito Relevante'
Train_MR = train.loc[filtra_Train_MR,:]

Train = [Train_MI,Train_I,Train_N,Train_R,Train_MR]

Train_MI

,Tweets,Classificação
0,@tavaresqq a sua troca deu certo? precisando é...,Muito Irrelevante
3,oi @magazineluiza o @luansantana tá há dois m...,Muito Irrelevante
6,@mateus_didi @americanascom @magazineluiza @re...,Muito Irrelevante
12,"rt @heartilyshawn: olá, @magazineluiza o canto...",Muito Irrelevante
14,@odeiopepino @detalhesslr @americanascom @maga...,Muito Irrelevante
...,...,...
741,"@sandrac58953439 oi, sandra. sabia que agora t...",Muito Irrelevante
743,@magazineluiza oi luciana legal lugar como voc...,Muito Irrelevante
744,@magazineluiza magalu vc é muito gostosa,Muito Irrelevante
747,@elisavitorio oi. peço desculpa por isso. 😔 me...,Muito Irrelevante


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test['Classificação'] = test['Classificação'].astype('category')
test.Classificação.cat.categories = ['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante']
test.Classificação.cat = pd.Categorical(test.Classificação, categories=['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante'], ordered=True)
test.head(5)

,Tweets,Classificação
0,@magazineluiza aquele que eh quero twr e nao t...,Nulo
1,@teeeeeeiii @magazineluiza kkkkk o coração qua...,Irrelevante
2,@onofrifernando oi! te chamei no dm da uma olh...,Muito Irrelevante
3,@felliperadama @americanascom @magazineluiza @...,Irrelevante
4,@robrtplant @magazineluiza bom dia! informamos...,Nulo


In [6]:
filtra_Test_MI = test['Classificação'] == 'Muito Irrelevante'
Test_MI = test.loc[filtra_Test_MI,:] 
filtra_Test_I = train['Classificação'] == 'Irrelevante'
Test_I = test.loc[filtra_Test_I,:]
filtra_Test_N = train['Classificação'] == 'Nulo'
Test_N = test.loc[filtra_Test_N,:]
filtra_Test_R = train['Classificação'] == 'Relevante'
Test_R = train.loc[filtra_Test_R,:]
filtra_Test_MR = train['Classificação'] == 'Muito Relevante'
Test_MR = test.loc[filtra_Test_MR,:]
Test = [Test_I,Test_MI,Test_N,Test_R,Test_MR]

___
## Classificador automático de sentimento


Nosso produto é a **Magazine Luiza**, uma empresa varejista de eletrônicos e imóveis. Possui lojas físicas, mas tem maior notoriedade em sua plataforma digital, cujas críticas e elogios estão fortemente presente na conta do Tweeter da empresa.

Para a classificação dos tweets envolvendo a Magazine Luiza, consideramos:
   * **“Muito relevantes”** aqueles que possuem problemas ou elogios à marca, detalhando o motivo de sua crítica 
   * **“Relevantes”** os menos detalhados.
   * **“Nulo”** são aqueles que demandam cupons de desconto para a marca e tweets com respostas à própria marca, que aparecem, na maioria das vezes, sem contexto. 
   * **“Irrelevantes”** foram os comentários ofensivos, que mostravam indignação com a marca, mas utilizando apenas palavras de baixo calão.
   * **“Muito irrelevante”** foi marcado para os tweets que a própria marca fazia, respondendo os clientes ou para os tweets que não tinham nada a ver com a marca tampouco com o que ela propõe.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
#define a função para tratar os dados 
def cleanup(tweet):
    #cria uma lista para a lista de palavaras filtradas
    tweet_limpo = []
    #definição dos métodos de tratamento do dataframe
    punctuation = '[@!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    stopwords = nltk.corpus.stopwords.words('portuguese') 
    text_subbed = re.sub(pattern, '', tweet)
    #Limpa stopwords
    token_words = TweetTokenizer().tokenize(text_subbed)
    for word in token_words:
        if word not in stopwords:
            tweet_limpo.append(word)
    return tweet_limpo

In [8]:
#Limpa os tweets de acordo com  a sua relevância 
Train_Limpo = []
for dataframe in Train:
    lista_de_tweets = dataframe['Tweets'].values.tolist() 
    lista_tweets_limpos =[]
    for tweet in lista_de_tweets:
        train_TwL = cleanup(tweet)
        lista_tweets_limpos.extend(train_TwL)
    Train_Limpo.append(lista_tweets_limpos)
    
#Calcula e aloca em variável a probabilidade de ser relevante
prob_Re = train['Classificação'].value_counts(True, sort=False)
P_MI = prob_Re[0]
P_I = prob_Re[1]
P_N = prob_Re[2]
P_R = prob_Re[3]
P_MR = prob_Re[4]

P_Re = [P_MI,P_I,P_N,P_R,P_MR]

#Separa as listas provevineites dos dataframes
palavras_train_MI = pd.Series(Train_Limpo[0]).value_counts().to_dict()
palavras_train_I = pd.Series(Train_Limpo[1]).value_counts().to_dict()
palavras_train_N = pd.Series(Train_Limpo[2]).value_counts().to_dict()
palavras_train_R = pd.Series(Train_Limpo[3]).value_counts().to_dict()
palavras_train_MR = pd.Series(Train_Limpo[4]).value_counts().to_dict()

dic_Fa_palavras=[palavras_train_MI, palavras_train_I,palavras_train_N,palavras_train_R,palavras_train_MR] 


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/DuduSantoro/nltk_data'
    - '/opt/anaconda3/nltk_data'
    - '/opt/anaconda3/share/nltk_data'
    - '/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
def Naive_Bayes(tweet):
    clean_tweet = cleanup(tweet)
    serie_de_fa =[]
    for dic in dic_Fa_palavras:
        dic_f ={}
        for palavra in clean_tweet:
            if palavra in dic:
                dic_f[palavra] = dic[palavra]
            if palavra not in dic:
                dic_f[palavra] = 0
        serie_de_fa.append(pd.Series(dic_f))
        
    fa_palavras_MI = serie_de_fa[0]
    fa_palavras_I = serie_de_fa[1]
    fa_palavras_N = serie_de_fa[2]
    fa_palavras_R = serie_de_fa[3]
    fa_palavras_MR = serie_de_fa[4]
    fa_palavras_Re = [fa_palavras_MI,fa_palavras_I,fa_palavras_N,fa_palavras_R,fa_palavras_MR]  
    lista_prob = []
    alpha = 0.01
    for i in range(0,len(fa_palavras_Re)):
        contagem_palavra_mais_alpha = fa_palavras_Re[i].add(alpha)
        contagem_todas_palavras = sum(dic_Fa_palavras[i].values())
        laplace_smothing = contagem_palavra_mais_alpha.divide(contagem_todas_palavras + alpha*(len(dic_Fa_palavras[i])))
        smoothing = np.log10(laplace_smothing)
        P_palavras_dado_Re = smoothing.sum()
        lista_prob.append(P_palavras_dado_Re + (np.log10(P_Re[i])))
    MAX = 0
    for k in range(0,len(lista_prob)):
        if lista_prob[k] >= lista_prob[MAX]:
            MAX = k
    if MAX == 0:
        Relevancia_NV = 'Muito Irrelevante'
    if MAX == 1:
        Relevancia_NV = 'Irrelevante'
    if MAX == 2:
        Relevancia_NV = 'Nulo'
    if MAX == 3:
        Relevancia_NV = 'Relevante'
    if MAX == 4:
        Relevancia_NV = 'Muito Relevante'

    return Relevancia_NV

In [ ]:
Cat_NaiveBayes_train = []
for i in train['Tweets']:
    Cat_NaiveBayes_train.append(Naive_Bayes(i))
train['Relevancia_NB'] = Cat_NaiveBayes_train
train


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
Cat_NaiveBayes_test = []
for i in test['Tweets']:
    Cat_NaiveBayes_test.append(Naive_Bayes(i))
test['Relevancia_NB'] = Cat_NaiveBayes_test
test

___
### Concluindo

In [ ]:
relevancia = ['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante']
Relevancia_NB_X_Classificacao_train = pd.crosstab(train.Classificação,train.Relevancia_NB,normalize=True)[relevancia]
Relevancia_NB_X_Classificacao_train

In [ ]:
print(f'A proporção de acerto do nosso classificador é {(Relevancia_NB_X_Classificacao_train.iloc[0,0]+Relevancia_NB_X_Classificacao_train.iloc[1,1]+Relevancia_NB_X_Classificacao_train.iloc[2,2]+Relevancia_NB_X_Classificacao_train.iloc[3,3]+Relevancia_NB_X_Classificacao_train.iloc[4,4])*100:.5g}%')

In [10]:
Relevancia_NB_X_Classificacao_test = pd.crosstab(test.Classificação,test.Relevancia_NB,normalize= True)[relevancia]
Relevancia_NB_X_Classificacao_test

AttributeError: 'DataFrame' object has no attribute 'Relevancia_NB'

In [ ]:
Efetividade_por_categoria_test = pd.crosstab(test.Classificação,test.Relevancia_NB,normalize='index')[relevancia]
Efetividade_por_categoria_test

In [9]:
print(f'A proporção de acerto do nossvo classificador é {(Relevancia_NB_X_Classificacao_test.iloc[0,0]+Relevancia_NB_X_Classificacao_test.iloc[1,1]+Relevancia_NB_X_Classificacao_test.iloc[2,2]+Relevancia_NB_X_Classificacao_test.iloc[3,3]+Relevancia_NB_X_Classificacao_test.iloc[4,4])*100:.5g}%')
print(f'A proporção de acerto da categoria Muito Irrelevante é {Efetividade_por_categoria_test.iloc[0,0]*100:.5g}% e de erros eh {(Efetividade_por_categoria_test.iloc[0,1]+Efetividade_por_categoria_test.iloc[0,2]+Efetividade_por_categoria_test.iloc[0,3]+Efetividade_por_categoria_test.iloc[0,4])*100:.5g}%') 
print(f'A proporção de acerto da categoria Irrelevante é {Efetividade_por_categoria_test.iloc[1,1]*100:.5g}% e de erros eh {(Efetividade_por_categoria_test.iloc[1,0]+Efetividade_por_categoria_test.iloc[1,2]+Efetividade_por_categoria_test.iloc[1,3]+Efetividade_por_categoria_test.iloc[1,4])*100:.5g}%')
print(f'A proporção de acerto da categoria Nulo é {Efetividade_por_categoria_test.iloc[2,2]*100:.5g}% e de erros eh {(Efetividade_por_categoria_test.iloc[2,1]+Efetividade_por_categoria_test.iloc[2,0]+Efetividade_por_categoria_test.iloc[2,3]+Efetividade_por_categoria_test.iloc[2,4])*100:.5g}%')
print(f'A proporção de acerto da categoria Relevante é {Efetividade_por_categoria_test.iloc[3,3]*100:.5g}% e de erros eh {(Efetividade_por_categoria_test.iloc[3,1]+Efetividade_por_categoria_test.iloc[3,2]+Efetividade_por_categoria_test.iloc[3,0]+Efetividade_por_categoria_test.iloc[3,4])*100:.5g}%')
print(f'A proporção de acerto da categoria Muito Relevante é {Efetividade_por_categoria_test.iloc[4,4]*100:.5g}% e de erros eh {(Efetividade_por_categoria_test.iloc[4,1]+Efetividade_por_categoria_test.iloc[4,2]+Efetividade_por_categoria_test.iloc[4,3]+Efetividade_por_categoria_test.iloc[4,0])*100:.5g}%')

NameError: name 'Relevancia_NB_X_Classificacao_test' is not defined

In [ ]:
amostra_train = pd.DataFrame()
amostra_train['Treinamento Naive Bayes'] = train.loc[:,'Relevancia_NB'].value_counts(normalize = 'index')
amostra_train['Treinamento'] = train.loc[:,'Classificação'].value_counts(normalize = 'index')
amostra_test = pd.DataFrame()
amostra_test['Teste Naive Bayes'] = test.loc[:,'Relevancia_NB'].value_counts(normalize = 'index')
amostra_test['Teste'] = test.loc[:,'Classificação'].value_counts(normalize = 'index')

fig = plt.figure(figsize=(20, 20))
ax1 =fig.add_subplot(221)
ax2 =fig.add_subplot(222)

analise_train = amostra_train.plot.bar(ax=ax1)
analise_test = amostra_test.plot.bar(ax=ax2)


___
### Plano de Extensão

Como plano de expansão para a melhoria do nosso classificador, pretendemos diferenciar os empecilhos do clientes, pois ao longo do projeto, percebemos que as críticas são divididas por: aqueles insatisfeitos com o serviço de entrega dos produtos da Magazine Luiza, e aqueles insatisfeitos com o serviço de atendimento online, ou seja, com a própria conta de Tweeter da marca. Basicamente, dentro das categorias "Muito relevante" e "Relevante", criaremos duas subcategorias, que serão os dois tipos de críticas em relação à marca.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**